# 命名实体识别

## 一、演练介绍
### 1. 演练内容
本演练中，将演练如何从不规则的文本中提取信息，将信息规则化，从而构建可以生成知识图谱的数据。
在羡慕，演练如何从文本中识别命名实体。

### 2. 演练技能点
在本演练中，将涉及
- 命名实体识别任务的定义与理论解决方法
- 命名实体识别任务中的数据集处理方法
- 命名实体识别模型——BiLstm-CRF 的理论知识
- 用 Keras 构建 BiLstm-CRF 模型
- BiLstm-CRF 的训练与预测方法

### 3. 演练要求
本演练要求具备以下基本能力：
- Keras 的基本使用方法
- 一定的数学基础


## 二、演练原理

### 1. 命名实体识别任务
命名实体识别是自然语言处理中的一项基础任务，命名实体指文本中具有特定意义的实体，通常包括人名，地名，专有名词等。

如在文本“张无忌，金庸武侠小说《倚天屠龙记》人物角色，中土明教第三十四代教主。武当七侠之一张翠山与天鹰教紫微堂主殷素素之子，明教四大护教法王之一金毛狮王谢逊义子。” 中，
- 人名实体有：张无忌，张翠山，殷素素，谢逊
- 书名实体有：倚天屠龙记
- 门派实体有：明教，武当，天鹰教

因此，命名实体识别任务通常包括两部分
- 实体的边界识别：如正确识别“张翠山”，而不是“张翠”或“张翠山与”等
- 确定实体的类型：如张无忌为人名实体，而不是门派实体或者书名实体

命名实体的标注方法有多种，在本演练中使用 BMEO 标注方法
- B 实体词首
- M 实体词中
- E 实体词尾
- O 非实体

结合实体类型，“武当七侠之一张翠山与天鹰教紫微堂主殷素素之子” 这份文本就会被标注为“武/门派_B 当/门派_E 七/O 侠/O 之/O 一/O 张/人名_B 翠/人名_M 山/人名_E 与/O 天/门派_B 鹰/门派_M 教/门派_E 紫/O 微/O 堂/O 主/O 殷/人名_B 素/人名_M 素/人名_E 之/O 子/O”



### <a id="序列标注">2. 序列标注</a>
命名实体识别任务本质上是一个序列标注问题，即给序列中的每一帧进行分类，在这里每一帧代表一个字。如”武当七侠之一张翠山“，不考虑实体类型，则共有四个标签 BMEO。既然是分类问题，就很自然地想到逐帧分类：即训练一个判别器，输入一个字，输出该字的类别，如下图所示

![softmax.png](01.png)

但是实际上，并不是说“张”这个字一定代表实体词首，有可能是“张开”这个词的起始，但“张开”并非实体。因此，每一帧都是上下文关联的，如“张”后面跟着“翠山”，那么“张”就是实体词首，反之则不一定，如下图所示

![crf.png](02.png)


同时目标输出序列本身会带有一些上下文的关联，比如实体词尾前一帧不可能是非实体，实体词中后一帧要么是实体词中要么是实体词尾。这与普通的分类任务不同。如果一个输入有 $n$ 帧，每一帧的标签有 $k$ 种可能性，那么理论上就有 $k^n$ 种不同的输出。如下图所示，每一个点代表一个标签的可能性，点之间的连线表示标签之间的关联，而每一条完整的路径，表示一种输出。

![graph.png](3.png)

综上所述，逐帧分类是将序列标注看成 $n$ 个 $k$ 分类问题，而真正的序列标注是 1 个 $k^n$ 分类问题。

### 3. 条件随机场
条件随机场（conditional random field，简称 CRF），是是一种鉴别式机率模型。在序列标注问题中，我们要计算的是条件概率 $P(y_1,...,y_n|x), x=(x_1,...,x_n)$ 。结合前面[序列标注](#序列标注)的内容，可以理解为给予图中的边以权重，并找到权重最高的一条路径作为输出。
CRF 中定义了特征函数来给边赋予权重，特征函数定义为 $f(s,i,l_i,l_{i-1})$ 
- $s$ 输入的句子
- $i$ 句子 $s$ 中第 $i$ 个词
- $l_i$ 第 $i$ 个词的标签
- $l_i-1$ 第 $i-1$ 个词的标签

定义好特征函数后，给每一个特征函数$f_j$赋予一个权重$\lambda_j$，从而，对于输入 $s$ 和 标注序列 $l$，$l$ 的评分为

$score(l|s)=\sum^m_{j=1}\sum^n_{i=1}\lambda_j f_j(s,i,l_i,l_{i-1})$

对分数进行指数化和标准化，就可以得到标注序列 $l$ 的概率值 $p(l|s)$，如下所示

$p(l|s)=\frac{\exp[score(l|s)]}{\sum_{l'}\exp[score(l'|s)]}$

### 4. BiLSTM-CRF
长短期记忆网络（Long Short-Term Memory，简称 LSTM），是循环神经网络（Recurrent Neural Network，简称 RNN）的一种，BiLSTM 是由前向 LSTM 与后向 LSTM 组合而成，由于其设计的特点，在自然语言处理任务中都常被用来建模上下文信息。 

与 CRF 不同的是，BiLSTM 依靠神经网络超强的非线性拟合能力，在训练时将数据变换到高维度的非线性空间中去，从而学习出一个模型。虽然 BiLSTM 的精度非常的高，但是在预测时，会出现一些明显的错误，如实体词尾后一帧依然预测为实体词尾等，而在 CRF 中，因为特征函数的存在，限定了标签之间的关系。因此，将 CRF 接到 BiLSTM 上，就可以将两者的特点结合，取长补短，通过 BiLSTM 提取高效的特征，让 CRF 的学习更加有效。


## 三、演练步骤
先安装需要的包：

In [10]:
# 使用前一个演练的 tf 和 keras 即可
#!pip install tensorflow==1.13.1
#!pip install keras==2.2.4

### 1. 数据处理

原始文本和标签分别定义为

In [11]:
raw_text = '''张无忌，金庸武侠小说《倚天屠龙记》人物角色，中土明教第三十四代教主。武当七侠之一张翠山与天鹰教紫微堂主殷素素之子，明教四大护教法王之一金毛狮王谢逊义子。
              张翠山，《倚天屠龙记》第一卷的男主角，在武当七侠之中排行第五，人称张五侠。与天鹰教殷素素结为夫妇，生下张无忌，后流落到北极冰海上的冰火岛，与谢逊相识并结为兄弟。
              殷素素，金庸武侠小说《倚天屠龙记》第一卷的女主人公。天鹰教紫薇堂堂主，容貌娇艳无伦，智计百出，亦正亦邪。与武当五侠张翠山同赴王盘山，结果被金毛狮王谢逊强行带走，三人辗转抵达冰火岛。殷素素与张翠山在岛上结为夫妇，并诞下一子张无忌。
              谢逊，是金庸武侠小说《倚天屠龙记》中的人物，字退思，在明教四大护教法王中排行第三，因其满头金发，故绰号“金毛狮王”。
           '''
annotations = {'name':['张无忌','张翠山','殷素素','谢逊'], 'book':['倚天屠龙记'],'org':['明教','武当','天鹰教']}
raw_text, annotations

('张无忌，金庸武侠小说《倚天屠龙记》人物角色，中土明教第三十四代教主。武当七侠之一张翠山与天鹰教紫微堂主殷素素之子，明教四大护教法王之一金毛狮王谢逊义子。\n              张翠山，《倚天屠龙记》第一卷的男主角，在武当七侠之中排行第五，人称张五侠。与天鹰教殷素素结为夫妇，生下张无忌，后流落到北极冰海上的冰火岛，与谢逊相识并结为兄弟。\n              殷素素，金庸武侠小说《倚天屠龙记》第一卷的女主人公。天鹰教紫薇堂堂主，容貌娇艳无伦，智计百出，亦正亦邪。与武当五侠张翠山同赴王盘山，结果被金毛狮王谢逊强行带走，三人辗转抵达冰火岛。殷素素与张翠山在岛上结为夫妇，并诞下一子张无忌。\n              谢逊，是金庸武侠小说《倚天屠龙记》中的人物，字退思，在明教四大护教法王中排行第三，因其满头金发，故绰号“金毛狮王”。\n           ',
 {'name': ['张无忌', '张翠山', '殷素素', '谢逊'],
  'book': ['倚天屠龙记'],
  'org': ['明教', '武当', '天鹰教']})

#### 1.1将标注转换为 BMEO 格式

In [12]:
import re

# 先去掉原始文本中的换行和空格符
raw_text = raw_text.replace('\n', '').replace(' ', '')
# 初始化 label：将其全部初始化为 O
labels = len(raw_text)*['O']

# 通过 key-value 的方式遍历 annotations 字典，进行转换
for ann, entities in annotations.items():
    for entity in entities:
        # 先生成实体对应的 BME 标注类型
        B, M, E = [['{}_{}'.format(ann,i)] for i in ['B','M','E']]
        # 计算实体词中的数量
        M_len = len(entity) - 2
        # 生成 label，如果词中数为0，则直接为 BE，不然按数量添加 M
        label = B + M * M_len + E if M_len else B + E
        # 从原始文本中找到实体对应出现的所有位置
        idxs = [r.start() for r in re.finditer(entity, raw_text)]
        
        for idx in idxs:
        # 替换原 label 中的 O 为实际 label
            labels[idx:idx+len(entity)] = label


# 打印原始文本和对应转换后的 label
for ann,label in zip(raw_text,labels):
    print(ann, label)

张 name_B
无 name_M
忌 name_E
， O
金 O
庸 O
武 O
侠 O
小 O
说 O
《 O
倚 book_B
天 book_M
屠 book_M
龙 book_M
记 book_E
》 O
人 O
物 O
角 O
色 O
， O
中 O
土 O
明 org_B
教 org_E
第 O
三 O
十 O
四 O
代 O
教 O
主 O
。 O
武 org_B
当 org_E
七 O
侠 O
之 O
一 O
张 name_B
翠 name_M
山 name_E
与 O
天 org_B
鹰 org_M
教 org_E
紫 O
微 O
堂 O
主 O
殷 name_B
素 name_M
素 name_E
之 O
子 O
， O
明 org_B
教 org_E
四 O
大 O
护 O
教 O
法 O
王 O
之 O
一 O
金 O
毛 O
狮 O
王 O
谢 name_B
逊 name_E
义 O
子 O
。 O
张 name_B
翠 name_M
山 name_E
， O
《 O
倚 book_B
天 book_M
屠 book_M
龙 book_M
记 book_E
》 O
第 O
一 O
卷 O
的 O
男 O
主 O
角 O
， O
在 O
武 org_B
当 org_E
七 O
侠 O
之 O
中 O
排 O
行 O
第 O
五 O
， O
人 O
称 O
张 O
五 O
侠 O
。 O
与 O
天 org_B
鹰 org_M
教 org_E
殷 name_B
素 name_M
素 name_E
结 O
为 O
夫 O
妇 O
， O
生 O
下 O
张 name_B
无 name_M
忌 name_E
， O
后 O
流 O
落 O
到 O
北 O
极 O
冰 O
海 O
上 O
的 O
冰 O
火 O
岛 O
， O
与 O
谢 name_B
逊 name_E
相 O
识 O
并 O
结 O
为 O
兄 O
弟 O
。 O
殷 name_B
素 name_M
素 name_E
， O
金 O
庸 O
武 O
侠 O
小 O
说 O
《 O
倚 book_B
天 book_M
屠 book_M
龙 book_M
记 book_E
》 O
第 O
一 O
卷 O
的 O
女 O
主 O
人 O
公 O
。 O
天 org_B
鹰 org_M


#### 1.2 数据集预处理
在自然语言处理中，需要将文本数据特征提取为向量数据，才能被模型使用。在本演练中使用的是词袋模型，忽略文本的语法和语序要素，将其仅仅看做是若干个词汇的集合。

In [13]:
from collections import Counter
import numpy as np
from keras.preprocessing.sequence import pad_sequences

先统计训练集中每个字出现的次数，然后建立字典表，只记录出现次数不小于 2 的字

In [14]:
# 统计每个字出现的次数
word_counts = Counter(raw_text)
# 建立字典表，只记录出现次数不小于 2 的字
vocab = [w for w, f in iter(word_counts.items()) if f >= 2]
word_counts, vocab

(Counter({'张': 8,
          '无': 4,
          '忌': 3,
          '，': 21,
          '金': 7,
          '庸': 3,
          '武': 6,
          '侠': 7,
          '小': 3,
          '说': 3,
          '《': 4,
          '倚': 4,
          '天': 7,
          '屠': 4,
          '龙': 4,
          '记': 4,
          '》': 4,
          '人': 5,
          '物': 2,
          '角': 2,
          '色': 1,
          '中': 4,
          '土': 1,
          '明': 3,
          '教': 9,
          '第': 5,
          '三': 3,
          '十': 1,
          '四': 3,
          '代': 1,
          '主': 5,
          '。': 9,
          '当': 3,
          '七': 2,
          '之': 4,
          '一': 5,
          '翠': 4,
          '山': 5,
          '与': 5,
          '鹰': 3,
          '紫': 2,
          '微': 1,
          '堂': 3,
          '殷': 4,
          '素': 8,
          '子': 3,
          '大': 2,
          '护': 2,
          '法': 2,
          '王': 6,
          '毛': 3,
          '狮': 3,
          '谢': 4,
          '逊': 4,
          '义': 1,
         

原始的数据集是字符串格式的，每句话用句号隔开，在训练过程中，需要把每句话拆开作为一个样本，因为每句话的长度不同，所以要定义一个最大长度，对于小于这个最大长度的句子，在左边或者右边填充固定的数字。

In [15]:
label_set = list(set(labels))

# 拆分训练集，每一句话作为一个样本，先找到每个句号的位置
sentence_len = [r.start()+1 for r in re.finditer('。', raw_text)]

# 进行拆分，这里要注意最后一个句号后面不需要拆分，所以最后一个位置不需要取到
split_text = np.split(list(raw_text), sentence_len[:-1])
split_label = np.split(labels, sentence_len[:-1])
split_text, split_label 

([array(['张', '无', '忌', '，', '金', '庸', '武', '侠', '小', '说', '《', '倚', '天',
         '屠', '龙', '记', '》', '人', '物', '角', '色', '，', '中', '土', '明', '教',
         '第', '三', '十', '四', '代', '教', '主', '。'], dtype='<U1'),
  array(['武', '当', '七', '侠', '之', '一', '张', '翠', '山', '与', '天', '鹰', '教',
         '紫', '微', '堂', '主', '殷', '素', '素', '之', '子', '，', '明', '教', '四',
         '大', '护', '教', '法', '王', '之', '一', '金', '毛', '狮', '王', '谢', '逊',
         '义', '子', '。'], dtype='<U1'),
  array(['张', '翠', '山', '，', '《', '倚', '天', '屠', '龙', '记', '》', '第', '一',
         '卷', '的', '男', '主', '角', '，', '在', '武', '当', '七', '侠', '之', '中',
         '排', '行', '第', '五', '，', '人', '称', '张', '五', '侠', '。'], dtype='<U1'),
  array(['与', '天', '鹰', '教', '殷', '素', '素', '结', '为', '夫', '妇', '，', '生',
         '下', '张', '无', '忌', '，', '后', '流', '落', '到', '北', '极', '冰', '海',
         '上', '的', '冰', '火', '岛', '，', '与', '谢', '逊', '相', '识', '并', '结',
         '为', '兄', '弟', '。'], dtype='<U1'),
  array(['殷', '素', '素', '，', '金', 

In [16]:
# 构建词袋模型，这里要将字典从 2 开始编号，把 0 和 1 空出来，0 作为填充元素，1 作为不在字典中的字的编号
word2idx = dict((w,i+2) for i,w in enumerate(vocab))
label2idx = [[label_set.index(w) for w in s] for s in split_label]
word2idx, label2idx

({'张': 2,
  '无': 3,
  '忌': 4,
  '，': 5,
  '金': 6,
  '庸': 7,
  '武': 8,
  '侠': 9,
  '小': 10,
  '说': 11,
  '《': 12,
  '倚': 13,
  '天': 14,
  '屠': 15,
  '龙': 16,
  '记': 17,
  '》': 18,
  '人': 19,
  '物': 20,
  '角': 21,
  '中': 22,
  '明': 23,
  '教': 24,
  '第': 25,
  '三': 26,
  '四': 27,
  '主': 28,
  '。': 29,
  '当': 30,
  '七': 31,
  '之': 32,
  '一': 33,
  '翠': 34,
  '山': 35,
  '与': 36,
  '鹰': 37,
  '紫': 38,
  '堂': 39,
  '殷': 40,
  '素': 41,
  '子': 42,
  '大': 43,
  '护': 44,
  '法': 45,
  '王': 46,
  '毛': 47,
  '狮': 48,
  '谢': 49,
  '逊': 50,
  '卷': 51,
  '的': 52,
  '在': 53,
  '排': 54,
  '行': 55,
  '五': 56,
  '结': 57,
  '为': 58,
  '夫': 59,
  '妇': 60,
  '下': 61,
  '冰': 62,
  '上': 63,
  '火': 64,
  '岛': 65,
  '并': 66,
  '亦': 67},
 [[8,
   2,
   5,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   1,
   3,
   3,
   3,
   6,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   0,
   9,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4],
  [0,
   9,
   4,
   4,
   4,
   4,
   8,
   2,
   5,
   4,
   0,
   7,
   9

In [17]:
# 构建输入，即对于样本中每一个字，从词袋模型中找到这个字对应的 idx，出现频率过低的字，并没有出现在词袋模型中，此时将这些字的 idx 取为 1
train_x = [[word2idx.get(w, 1) for w in s] for s in split_text]

max_len = 64

# 在输入的左边填充 0，在输出的左端填充-1
train_x = pad_sequences(train_x, max_len, value=0)
train_y = pad_sequences(label2idx, max_len, value=-1)
train_y = np.expand_dims(train_y, 2)
train_x.shape, train_y.shape

((9, 64), (9, 64, 1))

### 2. 构建网络模型
在 Keras 中，已经包含了 BiLSTM 模型中的各个组件，只需导入构建就可以了，而 CRF 层需要导入第三方库 keras-contrib 来使用。

首先安装 keras-contrib

方法如下：

1. 下载 https://codeload.github.com/keras-team/keras-contrib/zip/refs/heads/master
2. 解压，并执行： python setup.py install

然后，修改：C:\Dev\Python\venvs\py37kg01\Lib\site-packages\keras_contrib-2.0.8-py3.7.egg\keras_contrib\layers\ctf.py 

即：keras_contrib.layers.crf.py (对  K.slice 的调用，在 Line 463)

In [18]:
#! pip install git+https://www.github.com/keras-team/keras-contrib.git

In [19]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss

In [20]:
# 定义模型的超参
EMBED_DIM = 200
BiRNN_UNITS = 200

# 初始化模型
model = Sequential()
# 添加 Embedding 层，将输入转换成向量
model.add(Embedding(len(vocab)+2, EMBED_DIM, mask_zero=True))
# 添加 BiLstm 层
model.add(Bidirectional(LSTM(BiRNN_UNITS // 2, return_sequences=True)))
# 初始化 crf
crf = CRF(len(train_y), sparse_target=True)
# 将 crf 添加到模型中
model.add(crf)
model.summary()
# 编译模型
model.compile('adam', loss=crf_loss, metrics=[crf.accuracy])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         13600     
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 200)         240800    
_________________________________________________________________
crf_1 (CRF)                  (None, None, 9)           1908      
Total params: 256,308
Trainable params: 256,308
Non-trainable params: 0
_________________________________________________________________


f:\Venvs\kg01\lib\site-packages\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Cloning https://www.github.com/keras-team/keras-contrib.git to c:\users\lxmli\appdata\local\temp\pip-req-build-zqyo32je
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101662 sha256=647512ae8ff8dd4441e4a025ba0bbc9dd66b03de952a1c11d69b7f1486776879
  Stored in directory: C:\Users\lxmli\AppData\Local\Temp\pip-ephem-wheel-cache-3s9lm7nq\wheels\bb\1f\f2\b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git 'C:\Users\lxmli\AppData\Local\Temp\pip-req-build-zqyo32je'


### 3. 模型训练 

In [21]:
model.fit(train_x, train_y, batch_size=9, epochs=120)
model.save('model.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/120
9/9 [==============================] - 2s 206ms/step - loss: 4.0508 - crf_viterbi_accuracy: 0.0091
Epoch 2/120
9/9 [==============================] - 0s 7ms/step - loss: 4.0145 - crf_viterbi_accuracy: 0.0091
Epoch 3/120
9/9 [==============================] - 0s 7ms/step - loss: 3.9767 - crf_viterbi_accuracy: 0.0091
Epoch 4/120
9/9 [==============================] - 0s 6ms/step - loss: 3.9350 - crf_viterbi_accuracy: 0.0091
Epoch 5/120
9/9 [==============================] - 0s 6ms/step - loss: 3.8868 - crf_viterbi_accuracy: 0.0091
Epoch 6/120
9/9 [==============================] - 0s 6ms/step - loss: 3.8291 - crf_viterbi_accuracy: 0.0091
Epoch 7/120
9/9 [==============================] - 0s 7ms/step - loss: 3.7580 - crf_viterbi_accuracy: 0.0091
Epoch 8/120
9/9 [==============================] - 0s 8ms/step - loss: 3.6688 - crf_viterbi_accuracy: 0.2805
Epoch 9/120
9/9 [==============================] - 0s 8ms/step - loss: 3.5557 

### 4. 模型预测 
在这里使用训练集中出现的一句话进行预测

In [22]:
text = '谢逊，是金庸武侠小说《倚天屠龙记》中的人物，字退思，在明教四大护教法王中排行第三，因其满头金发，故绰号“金毛狮王"。'
text

'谢逊，是金庸武侠小说《倚天屠龙记》中的人物，字退思，在明教四大护教法王中排行第三，因其满头金发，故绰号“金毛狮王"。'

In [23]:
# 将预测数据转换为特征向量
pred_x = [word2idx.get(w, 1) for w in text]
pred_x = pad_sequences([pred_x], max_len)
pred_x

array([[ 0,  0,  0,  0,  0,  0, 49, 50,  5,  1,  6,  7,  8,  9, 10, 11,
        12, 13, 14, 15, 16, 17, 18, 22, 52, 19, 20,  5,  1,  1,  1,  5,
        53, 23, 24, 27, 43, 44, 24, 45, 46, 22, 54, 55, 25, 26,  5,  1,
         1,  1,  1,  6,  1,  5,  1,  1,  1,  1,  6, 47, 48, 46,  1, 29]])

In [24]:
# 使用模型进行预测
pred = model.predict(pred_x)
pred.shape

(1, 64, 9)

In [25]:
# 去除多余的维度
pred = np.squeeze(pred)[-len(text):]
pred.shape

(58, 9)

In [26]:
# 把输出向量转换为 label 对应的 idx
result = [np.argmax(r) for r in pred]
result

[8,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 1,
 3,
 3,
 3,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4]

In [27]:
# 打印输出结果
reslut_labels = [label_set[i] for i in result]
for w, l in zip(text, reslut_labels):
    print(w, l)

谢 name_B
逊 name_E
， O
是 O
金 O
庸 O
武 O
侠 O
小 O
说 O
《 O
倚 book_B
天 book_M
屠 book_M
龙 book_M
记 book_E
》 O
中 O
的 O
人 O
物 O
， O
字 O
退 O
思 O
， O
在 O
明 org_B
教 O
四 O
大 O
护 O
教 O
法 O
王 O
中 O
排 O
行 O
第 O
三 O
， O
因 O
其 O
满 O
头 O
金 O
发 O
， O
故 O
绰 O
号 O
“ O
金 O
毛 O
狮 O
王 O
" O
。 O


##  四、总结

在本演练中，了解了命名实体识别任务的定义，并用了 BiLstm-CRF 模型，结合简单的数据集实现了整个命名实体识别任务中的数据处理、训练与预测。
虽然在本演练中数据集规模较小，而且并没有切分验证集与测试集，但在实际的调试中，为了先确保数据管道与网络模型是否搭建正确，应先用少量数据集训练至过拟合，然后逐渐增大数据规模，并切分验证集和测试集，在验证集上对模型进行调优。

命名实体识别除了应用在构建知识图谱前期的数据结构化之外，还可以用于知识图谱构建完成后的智能问答上，如提问 “张无忌是谁？”，从中提取到人名实体“张无忌”，然后转化为 Cypher 语言 `match (n:角色) where n.name='张无忌' return n.desc` 进行查询，最后返回查询到的结果作为回答。

## 五、课后习题
1. 尝试扩充数据集，然后切分验证集与测试集进行训练。
2. 尝试使用函数式 API 构建 Bi-LSTM 模型。
3. 尝试将上述代码改为 tensorflow 2.x 的版本，尝试修改代码，并重新运行，实现上述功能
4. 思考是否有更简单的实现方式？